# Control Analysis

In this Jupyter notebook, the individual seeds for the control condition of each test (e.g. genome size, number of non-coding bases, etc.) is graphed out. 


In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import gc
from scipy import stats
from functools import reduce
from MyFunctions import read_fitness, read_bp, read_genes, read_robustness, perc_diff, name, mean_format, rank_sum_format, cond_color, seed_color, my_y_label

# Laptop
#INPUT_ROOT_DIR = "C://Users//davis//Dropbox//Freiburg Masters Semesters//Thesis//Results//Round02//"

# Desktop
INPUT_ROOT_DIR = "D://Dropbox_New//Dropbox//Freiburg Masters Semesters//Thesis//Results//Round02//"

OUTPUT_ROOT_DIR = INPUT_ROOT_DIR + "analysis//control_analysis//"
WINDOW_SIZE = 10000
GENOME_WINDOW_SIZE = 5000

print ("Setting up environment...")
# Make sure that the output directories exist and create them if not
if not os.path.exists(OUTPUT_ROOT_DIR):
    os.makedirs(OUTPUT_ROOT_DIR)
    
# stat_fitness_global
if not os.path.exists(OUTPUT_ROOT_DIR + "fitness_global//"):
    os.makedirs(OUTPUT_ROOT_DIR + "fitness_global//")
    
# stat_genes_global
if not os.path.exists(OUTPUT_ROOT_DIR + "genes_global//"):
    os.makedirs(OUTPUT_ROOT_DIR + "genes_global//")
if not os.path.exists(OUTPUT_ROOT_DIR + "genes_global//"):
    os.makedirs(OUTPUT_ROOT_DIR + "genes_global//")
    
# stat_genes_best (for non-coding)
if not os.path.exists(OUTPUT_ROOT_DIR + "non-coding//"):
    os.makedirs(OUTPUT_ROOT_DIR + "non-coding//")

if not os.path.exists(OUTPUT_ROOT_DIR + "robustness//"):
    os.makedirs(OUTPUT_ROOT_DIR + "robustness//")
    
if not os.path.exists(OUTPUT_ROOT_DIR + "evolvability//"):
    os.makedirs(OUTPUT_ROOT_DIR + "evolvability//")

# We need to be able to write out some statistics. Create an output file for this purpose.
LINE = "============================================="

plt.rc('legend',fontsize=20)
plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)

print("done.")

Setting up environment...
done.


## Read in the data for the control condition

In [14]:
print("Reading in control data...")

# stat_fitness_glob
df_seed01_control_fitness_global = read_fitness(INPUT_ROOT_DIR + "seed01//control//stats//stat_fitness_glob.out")
df_seed02_control_fitness_global = read_fitness(INPUT_ROOT_DIR + "seed02//control//stats//stat_fitness_glob.out")
df_seed03_control_fitness_global = read_fitness(INPUT_ROOT_DIR + "seed03//control//stats//stat_fitness_glob.out")
df_seed04_control_fitness_global = read_fitness(INPUT_ROOT_DIR + "seed04//control//stats//stat_fitness_glob.out")
df_seed05_control_fitness_global = read_fitness(INPUT_ROOT_DIR + "seed05//control//stats//stat_fitness_glob.out")

stat_fitness_glob_control = [df_seed01_control_fitness_global, df_seed02_control_fitness_global, df_seed03_control_fitness_global, df_seed04_control_fitness_global, df_seed05_control_fitness_global]

# stat_genes_global 
df_seed01_control_genes_global = read_genes(INPUT_ROOT_DIR + "seed01//control//stats//stat_genes_glob.out")
df_seed02_control_genes_global = read_genes(INPUT_ROOT_DIR + "seed02//control//stats//stat_genes_glob.out")
df_seed03_control_genes_global = read_genes(INPUT_ROOT_DIR + "seed03//control//stats//stat_genes_glob.out")
df_seed04_control_genes_global = read_genes(INPUT_ROOT_DIR + "seed04//control//stats//stat_genes_glob.out")
df_seed05_control_genes_global = read_genes(INPUT_ROOT_DIR + "seed05//control//stats//stat_genes_glob.out")

stat_genes_glob_control = [df_seed01_control_genes_global, df_seed02_control_genes_global, df_seed03_control_genes_global, df_seed04_control_genes_global, df_seed05_control_genes_global]

# stat_bp_best (for percent non-coding)
df_seed01_control_bp_best = read_bp(INPUT_ROOT_DIR + "seed01//control//stats//stat_bp_best.out")
df_seed02_control_bp_best = read_bp(INPUT_ROOT_DIR + "seed02//control//stats//stat_bp_best.out")
df_seed03_control_bp_best = read_bp(INPUT_ROOT_DIR + "seed03//control//stats//stat_bp_best.out")
df_seed04_control_bp_best = read_bp(INPUT_ROOT_DIR + "seed04//control//stats//stat_bp_best.out")
df_seed05_control_bp_best = read_bp(INPUT_ROOT_DIR + "seed05//control//stats//stat_bp_best.out")

stat_bp_best_control = [df_seed01_control_bp_best, df_seed02_control_bp_best, df_seed03_control_bp_best, df_seed04_control_bp_best, df_seed05_control_bp_best]

# stat_fitness_best (for percent non-coding)
df_seed01_control_fitness_best = read_fitness(INPUT_ROOT_DIR + "seed01//control//stats//stat_fitness_best.out")
df_seed02_control_fitness_best = read_fitness(INPUT_ROOT_DIR + "seed02//control//stats//stat_fitness_best.out")
df_seed03_control_fitness_best = read_fitness(INPUT_ROOT_DIR + "seed03//control//stats//stat_fitness_best.out")
df_seed04_control_fitness_best = read_fitness(INPUT_ROOT_DIR + "seed04//control//stats//stat_fitness_best.out")
df_seed05_control_fitness_best = read_fitness(INPUT_ROOT_DIR + "seed05//control//stats//stat_fitness_best.out")

stat_fitness_best_control = [df_seed01_control_fitness_best, df_seed02_control_fitness_best, df_seed03_control_fitness_best, df_seed04_control_fitness_best, df_seed05_control_fitness_best]

print("done.")

Reading in control data...
done.


# Graph results of stat_fitness_glob

The general idea is:

- For each column (e.g. genome size, fitness, etc.)
   - For each seed of the control  condition
      - Plot the individual seed
   - Plot the mean of all seeds for that aspect
   

In [19]:
# Allows one to just graph the last 50k generations, etc.
START_GEN = 1
STOP_GEN = 500000

invalid_headers = ['generation', 'secretion_error', 'parents_secretion_error', 'secretion_fitness', 'amt_compound_present', 'pop_size']

# For each column header that isn't invalid (e.g. genome size, metabolic error, etc.)
num_headers = len(df_seed01_control_fitness_global.columns) - len(invalid_headers)
for col_counter, col_header in enumerate(df_seed01_control_fitness_global.columns, -1):
        
    if col_header in invalid_headers:
        continue
    else:
        print(col_counter, '/',num_headers, ' - ', col_header)
        
        # Generate the generational indices for the DataFrame that will
        # store the mean values for each header
        df_control_mean = pd.DataFrame(df_seed01_control_fitness_global['generation'].copy())
                
        # Create a new figure for the current column
        fig = plt.figure(figsize=(30,20))
        ax = fig.add_subplot(111)
        fig.suptitle('Control - ' + name(col_header), x=0.5, y=0.92, fontsize=32)
        ax.set_xlabel("generation", fontsize=20)
        ax.set_ylabel(my_y_label(col_header),fontsize=20)
        
        # For each seed of the control condition
        for seed_counter, seed in enumerate(stat_fitness_glob_control):
                        
            # The name of the current seed
            seed_str = 'seed0'+str(seed_counter+1)
            
            # Smooth the seed's data with a rolling window
            df_control_mean[seed_str] = seed[col_header].rolling(WINDOW_SIZE).mean()
            
            # Add the current seed to the figure
            df_control_mean.iloc[START_GEN:STOP_GEN].plot(x='generation', y=seed_str, label=seed_str, color=seed_color(seed_str),  ax=ax)
        
                  
        #  Calculate the mean of the control condition and plot it to the figure
        df_control_mean['avg'] = df_control_mean.iloc[:,1:].mean(axis=1).rolling(WINDOW_SIZE).mean() # iloc[:,1:] to skip first column ('generation')
        df_control_mean.iloc[START_GEN:STOP_GEN].plot(x='generation', y='avg', label='control mean ' + name(col_header), linewidth=3.0, color='black', linestyle='dashed', ax=ax)
        if col_header == 'fitness':
            print("mean fitness: ", df_control_mean['avg'].iloc[START_GEN:STOP_GEN].mean(axis=0))
        fig.savefig(OUTPUT_ROOT_DIR+ 'fitness_global/control_' + col_header+'_b'+str(START_GEN)+'-e'+str(STOP_GEN)+'.png', bbox_inches='tight', pad_inches=0.25)
        fig.clear()
        plt.close(fig)
        
print("Done!")
 

1 / 5  -  fitness
mean fitness:  0.022725189396233487
2 / 5  -  genome_size
3 / 5  -  metabolic_error
4 / 5  -  parents_metabolic_error
5 / 5  -  metabolic_fitness
Done!


## Clean up the DataFrames

In [4]:
print("Cleaning up...")

for seed in stat_fitness_glob_control:
    del seed
    
del stat_fitness_glob_control, df_control_mean

gc.collect()
print("Done.")

Cleaning up...
Done.


## Graph out stat_genes_global

In [5]:
# Allows one to just graph the last 50k generations, etc.
START_GEN = 1
STOP_GEN = 500000

invalid_headers = ['generation']

# For each column header that isn't invalid (e.g. genome size, metabolic error, etc.)
num_headers = len(df_seed01_control_genes_global.columns) - len(invalid_headers)
for col_counter, col_header in enumerate(df_seed01_control_genes_global.columns):
        
    if col_header in invalid_headers:
        continue
    else:
        print(col_counter, '/',num_headers, ' - ', col_header)
        
        # Generate the generational indices for the DataFrame that will
        # store the mean values for each header
        df_control_mean = pd.DataFrame(df_seed01_control_genes_global['generation'].copy())
                
        # Create a new figure for the current column
        fig = plt.figure(figsize=(30,20))
        ax = fig.add_subplot(111)
        fig.suptitle('Control - ' + name(col_header), x=0.5, y=0.92, fontsize=32)
        ax.set_xlabel("generation", fontsize=20)
        ax.set_ylabel(my_y_label(col_header),fontsize=20)
        
        # For each seed of the control condition
        for seed_counter, seed in enumerate(stat_genes_glob_control):
                        
            # The name of the current seed
            seed_str = 'seed0'+str(seed_counter+1)
            
            # Smooth the seed's data with a rolling window
            df_control_mean[seed_str] = seed[col_header].rolling(WINDOW_SIZE).mean()
            
            # Add the current seed to the figure
            df_control_mean.iloc[START_GEN:STOP_GEN].plot(x='generation', y=seed_str, label=seed_str, color=seed_color(seed_str),  ax=ax)
        
             
        #  Calculate the mean of the control condition and plot it to the figure
        df_control_mean['avg'] = df_control_mean.iloc[:,1:].mean(axis=1).rolling(WINDOW_SIZE).mean() # iloc[:,1:] to skip first column ('generation')
        df_control_mean.iloc[START_GEN:STOP_GEN].plot(x='generation', y='avg', label='control mean ' + name(col_header), linewidth=3.0, color='black', linestyle='dashed', ax=ax)
        
        fig.savefig(OUTPUT_ROOT_DIR+ 'genes_global/control_' + col_header+'_b'+str(START_GEN)+'-e'+str(STOP_GEN)+'.png', bbox_inches='tight', pad_inches=0.25)
        fig.clear()
        plt.close(fig)
        
print("Done!")
 

1 / 8  -  num_coding_RNAs
2 / 8  -  num_non-coding_RNAs
3 / 8  -  avg_size_of_coding_RNAs
4 / 8  -  avg_size_of_non-coding_RNAs
5 / 8  -  num_functional_genes
6 / 8  -  num_non-functional_genes
7 / 8  -  avg_size_of_functional_genes
8 / 8  -  avg_size_of_non-functional_genes
Done!


## Clean up the DataFrames

In [6]:
print("Cleaning up...")

for seed in stat_genes_glob_control:
    del seed
    
del stat_genes_glob_control, df_control_mean
gc.collect()
print("Done.")

Cleaning up...
Done.


## Graph out the percent non-coding

This uses stat_bp_best and stat_fitness_best. We need the number of non-coding bp from stat_bp_best and the genome size from stat_fitness_best. 

In [42]:
print("Graphing percent non-coding...")

# Allows one to just graph the last 50k generations, etc.
START_GEN = 1
STOP_GEN = 500000

# Create a new figure to graph the results
fig = plt.figure(figsize=(30,20))
ax = fig.add_subplot(111)
fig.suptitle('Control - Percent Non-Coding', x=0.5, y=0.92, fontsize=32)
ax.set_xlabel("generation", fontsize=20)


# Copy the generational data
df_num_noncoding_control = pd.DataFrame(df_seed01_control_bp_best['generation'].copy())

# For each seed, find the number of bp not in any coding RNA into the results DataFrame
for seed_counter, seed in enumerate(stat_bp_best_control):
    seed_str = 'seed0' + str(seed_counter+1)
    df_num_noncoding_control[seed_str] = seed['num_bp_not_included_in_any_coding_RNA']
    
df_control_smoothed = pd.DataFrame(df_num_noncoding_control['generation'].copy())

# Now for each seed's number of BP not in any coding RNA, divide this by the total genome size
for seed_counter, seed in enumerate(stat_fitness_best_control):
    seed_str = 'seed0' + str(seed_counter+1)
    
    # Calculate the percent non-coding
    df_num_noncoding_control[seed_str] = 100*df_num_noncoding_control[seed_str] / seed['genome_size']
    
    # Smooth and plot the result
    df_control_smoothed[seed_str] = df_num_noncoding_control[seed_str].rolling(WINDOW_SIZE).mean()
    df_control_smoothed.iloc[START_GEN:STOP_GEN].plot(x='generation', y=seed_str, alpha=0.8, label=seed_str, color=seed_color(seed_str), ax=ax)
     

# Calculate the control average across all seeds
df_num_noncoding_control['avg'] = df_num_noncoding_control.iloc[:,1:].mean(axis=1).rolling(WINDOW_SIZE).mean()

# Plot the control average
df_num_noncoding_control.iloc[START_GEN:STOP_GEN].plot(x='generation', y='avg', label='control mean percent non-coding', color='black', linestyle='dashed', linewidth=3.0, ax=ax)


#Save and close the figure
fig.savefig(OUTPUT_ROOT_DIR+"non-coding/control_percent_non-coding_b"+str(START_GEN)+"-e"+str(STOP_GEN)+".png", bbox_inches='tight', pad_inches=0.25)
fig.clear()
plt.close(fig)

print("done.")
 
    

Graphing percent non-coding...
done.


## Clean up the DataFrames

In [8]:
print("Cleaning up...")

for df in stat_bp_best_control:
    del df

del stat_bp_best_control, df_num_noncoding_control
gc.collect()

print("done.")



Cleaning up...
done.


# Evolvability and Robustness

## Read in the data

In [20]:
print("Reading in data...")

# Control
df_seed01_control_robustness = read_robustness(INPUT_ROOT_DIR + "seed01//control//seed01_control_ancestor_robustness.out")
df_seed02_control_robustness = read_robustness(INPUT_ROOT_DIR + "seed02//control//seed02_control_ancestor_robustness.out")
df_seed03_control_robustness = read_robustness(INPUT_ROOT_DIR + "seed03//control//seed03_control_ancestor_robustness.out")
df_seed04_control_robustness = read_robustness(INPUT_ROOT_DIR + "seed04//control//seed04_control_ancestor_robustness.out")
df_seed05_control_robustness = read_robustness(INPUT_ROOT_DIR + "seed05//control//seed05_control_ancestor_robustness.out")

list_control_robustness = [df_seed01_control_robustness, df_seed02_control_robustness, df_seed03_control_robustness, df_seed04_control_robustness, df_seed05_control_robustness]

print("done.")

Reading in data...
done.


## Graph the robustness

Robustness in Aevol is the "percentage of neutral mutants".



In [10]:
print("Graphing robustness...")
# Allows one to just graph the last 50k generations, etc.
START_GEN = 1
STOP_GEN = 500000

# Plot out the results of all seeds for each column header
invalid_headers = ['generation']
num_headers = len(df_seed01_control_robustness.columns) - len(invalid_headers)
for col_counter, col_header in enumerate(df_seed01_control_robustness.columns):
    if col_header in invalid_headers:
        continue
    else:
        print(col_counter, '/',num_headers, ' - ', col_header)
        
        # Create new DataFrame for the control condition
        df_control_mean = pd.DataFrame(df_seed01_control_robustness['generation'].copy())
        df_control_smoothed = pd.DataFrame(df_seed01_control_robustness['generation'].copy())
        
        # Create a new figure for the current column
        fig = plt.figure(figsize=(30,20))
        ax = fig.add_subplot(111)
        fig.suptitle('Control - ' + name(col_header), x=0.5, y=0.92, fontsize=32)
        ax.set_xlabel("generation", fontsize=20)
        ax.set_ylabel(my_y_label(col_header),fontsize=20)
            
               
        # For each seed of the control condition
        for seed_counter, seed in enumerate(list_control_robustness):
            
            # Can't use a standard window (10k) because there are fewer data points in the robustness files
            ROBUSTNESS_WINDOW = int(len(seed.index)/20)
                        
            # The name of the current seed
            seed_str = 'seed0'+str(seed_counter+1)
            
            # Smooth the seed's data with a rolling window
            df_control_mean[seed_str] = seed[col_header]
            df_control_smoothed[seed_str] = df_control_mean[seed_str].rolling(ROBUSTNESS_WINDOW).mean()
                     
            # Add the current seed to the figure
            df_control_smoothed[df_control_smoothed['generation'].between(START_GEN,STOP_GEN)].plot(x='generation', y=seed_str, label=seed_str, color=seed_color(seed_str),  ax=ax)
            
                     
        #  Calculate the mean of the control condition seeds and plot it to the figure
        df_control_mean['avg'] = df_control_mean.iloc[:,1:].mean(axis=1).rolling(ROBUSTNESS_WINDOW).mean() # iloc[:,1:] to skip first column ('generation')
        df_control_mean[df_control_mean['generation'].between(START_GEN, STOP_GEN)].plot(x='generation', y='avg', label='control mean ' + name(col_header), linewidth=3.0, color='black', linestyle='dashed', ax=ax)
        
        fig.savefig(OUTPUT_ROOT_DIR+ 'robustness/control_' + col_header+'_b'+str(START_GEN)+'-e'+str(STOP_GEN)+'.png', bbox_inches='tight', pad_inches=0.25)
        fig.clear()
        plt.close(fig)


print("done.")

Graphing robustness...
1 / 12  -  frac_positive_offspring
2 / 12  -  frac_neutral_offspring
3 / 12  -  frac_neutral_mutants
4 / 12  -  frac_negative_offspring
5 / 12  -  cumul_delta-gap_positive_offspring
6 / 12  -  cumul_delta-gap_negative_offspring
7 / 12  -  delta-gap_best_offspring
8 / 12  -  delta-gap_worst_offspring
9 / 12  -  cumul_delta-fitness_positive_offspring
10 / 12  -  cumul_delta-fitness_negative_offspring
11 / 12  -  delta-fitness_best_offspring
12 / 12  -  delta-fitness_worst_offspring
done.


# Graph evolvability

Evolvability = (the fraction of positive offspring) * (the cumulative delta fitness gap of positive offspring).

In [40]:
print("Graphing evolvability...")
# Allows one to just graph the last 50k generations, etc.
START_GEN = 1
STOP_GEN = 500000
COLUMN_LABELS = ['generation','seed01', 'seed02', 'seed03', 'seed04', 'seed05'] # For resetting column labels after merging

# Create a new figure for the evolvability graph
fig = plt.figure(figsize=(30,20))
ax = fig.add_subplot(111)
fig.suptitle('Control - Evolvability', x=0.5, y=0.92, fontsize=32)
ax.set_xlabel("generation", fontsize=20)
ax.set_ylabel('evolvability',fontsize=20)


# Start a DataFrame for all of the evolvability data for each seed. Begin by copying its generational data.
df_control_evolvability = pd.DataFrame(df_seed01_control_robustness['generation'])

# For each seed of the control condition, calculate the evolvability.
# Evolvability = frac positive offspring * cumulative delta fitness gap of positive offspring
for seed_counter, seed in enumerate(list_control_robustness):
    
    # Make a temporary DataFrame to store the results of the calculation
    df_tmp =  pd.DataFrame(seed['frac_positive_offspring'] * seed['cumul_delta-fitness_positive_offspring'])
    df_tmp['generation'] = seed['generation'].copy() # Copy in the generational data
    
    # Merge the temp calculations and generational data
    df_control_evolvability = pd.merge(df_control_evolvability, df_tmp, on='generation', how='outer', sort=True)

# Fix the column headers
df_control_evolvability.columns=COLUMN_LABELS

# Now find the mean evolvability of the control condition over time
df_control_evolvability['avg'] = df_control_evolvability.iloc[:,1:].mean(axis=1)
#df_control_evolvability.to_csv("C://Users//Brian Davis//Desktop//evolvability.csv",sep='|')


# PLOT THE RESULTS OVER TIME

# For each seed, add it to the plot
for seed in df_control_evolvability.columns:
    if seed=='generation' or seed=='avg':
        continue
    else:
        df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].plot(x='generation', y=seed, label=seed, color=seed_color(seed), alpha=0.50, ax=ax)
        

# Plot the mean of the control condition
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].plot(x='generation', y='avg', label='control mean evolvability', color='black', linewidth=2.0, linestyle='dashed', ax=ax)

fig.savefig(OUTPUT_ROOT_DIR+ 'evolvability/control_evolvability_b'+str(START_GEN)+'-e'+str(STOP_GEN)+'.png', bbox_inches='tight', pad_inches=0.25)
fig.clear()
plt.close(fig)

# BOXPLOT
# Create a new figure for the boxplot
fig2 = plt.figure(figsize=(30,20))
ax2 = fig2.add_subplot(111)
fig2.suptitle('Control - Evolvability Boxplot', x=0.5, y=0.92, fontsize=32)
ax2.set_xlabel("generation", fontsize=20)
ax2.set_ylabel('evolvability',fontsize=20)        

seed_labels = ['seed01', 'seed02', 'seed03', 'seed04', 'seed05']        
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].boxplot(column=seed_labels, ax=ax2)

fig2.savefig(OUTPUT_ROOT_DIR + 'evolvability/control_evolvability_boxplot_b'+str(START_GEN)+'-e'+str(STOP_GEN)+'.png', bbox_inches='tight', pad_inches=0.25)
fig2.clear()
plt.close(fig2)

# HISTOGRAM
# Create a new figure for the histogram
fig3 = plt.figure(figsize=(30,20))
ax3_control_mean = plt.subplot(411)
ax3_control_seed01 = plt.subplot(423, sharex=ax3_control_mean, sharey=ax3_control_mean)
ax3_control_seed02 = plt.subplot(424, sharex=ax3_control_mean, sharey=ax3_control_mean)
ax3_control_seed03 = plt.subplot(425, sharex=ax3_control_mean, sharey=ax3_control_mean)
ax3_control_seed04 = plt.subplot(426, sharex=ax3_control_mean, sharey=ax3_control_mean)
ax3_control_seed05 = plt.subplot(427, sharex=ax3_control_mean, sharey=ax3_control_mean)

hist_axis_list = [ax3_control_mean, ax3_control_seed01, ax3_control_seed02, ax3_control_seed03, ax3_control_seed04, ax3_control_seed05]

# Plot the control condition to the histogram
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].hist(column=['seed01'], bins=20, alpha=0.50, ax=ax3_control_seed01)
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].hist(column=['seed02'], bins=20, alpha=0.50, ax=ax3_control_seed02)
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].hist(column=['seed03'], bins=20, alpha=0.50, ax=ax3_control_seed03)
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].hist(column=['seed04'], bins=20, alpha=0.50, ax=ax3_control_seed04)
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].hist(column=['seed05'], bins=20, alpha=0.50, ax=ax3_control_seed05)

# Plot the control condition to the histogram
df_control_evolvability[df_control_evolvability['generation'].between(START_GEN,STOP_GEN)].hist(column=['avg'], bins=20, alpha=0.50, ax=ax3_control_mean)

# Label the figure properly
fig3.suptitle('Control - Evolvability Histogram', x=0.5, y=0.92, fontsize=32)
# Set general parameters for all plots
for ax in hist_axis_list:
    ax.set_xlabel("Value", fontsize=18)
    ax.set_ylabel("Count", fontsize=18)
    ax.tick_params(axis='both', labelsize=20)

# Set individual labels
ax3_control_mean.set_title("Control Mean Evolvability (all seeds)", fontsize=25)
ax3_control_mean.xaxis.set_tick_params(labelbottom=True)

ax3_control_seed01.set_title("seed01", fontsize=20)
ax3_control_seed02.set_title("seed02", fontsize=20)
ax3_control_seed03.set_title("seed03", fontsize=20)
ax3_control_seed04.set_title("seed04", fontsize=20)
ax3_control_seed05.set_title("seed05", fontsize=20)

# Save and clear the figure
fig3.savefig(OUTPUT_ROOT_DIR + 'evolvability/control_evolvability_histogram_b'+str(START_GEN) + '-e' + str(STOP_GEN) + '.png', bbox_inches='tight', pad_inches=0.25)
fig3.clear()
plt.close(fig3)
print("done.")

Graphing evolvability...
done.


## Clean up the DataFrames

In [41]:
print("Cleaning up...")
for df in list_control_robustness:
    del df
    
del list_control_robustness, df_control_evolvability

print("done.")

Cleaning up...
done.
